In [1]:
%%capture
!python -m spacy download ru_core_news_lg 
!pip install natasha pandas numpy scikit-learn nltk
# !pip install -U 'spacy[transformers,lookups]'


In [2]:
# !pip install -U pip setuptools wheel
# !pip install spacy-huggingface-pipelines

## Что пробовать

- [x] Модель XX и использование английского дало неплохой результат **Стабильный результат**
- [X] Попробовать модель для русского языка **Получилось хуже**
- [] Аугментация **Результат**
- [x] Попробовать получать векторное представление [LLM](https://huggingface.co/ai-forever/sbert_large_nlu_ru) **Точно такой же как и spacy**
- [x] Попробовать различные комбинации вопросов 1 с 2, 1 с 3
- [] Посмотреть какие вопросы являются хорошими тригерами

In [3]:
import pandas as pd
import numpy as np
import re
import random 
import spacy
from spacy.training.example import Example
from tqdm import tqdm
import os
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, f1_score
import torch

import tensorflow as tf
# import autokeras as ak
from nltk.corpus import stopwords

# import mlflow
# from mlflow.models.signature import infer_signature
# from mlflow.tracking import MlflowClient
from itertools import combinations, combinations_with_replacement, permutations

import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)

from sklearn.utils import resample

2024-04-27 15:17:00.020000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 15:17:00.020145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 15:17:00.153017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Чтение данных

In [4]:
data_train = pd.read_csv("/kaggle/input/geek-bot-dataset/train_Feedback/train_data.csv")
data_train.head()

,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive
0,2024-04-01 08:30:00,Основы программирования,Интерактивный подход к изучению основ C# был о...,Нет.,Увеличить количество практических заданий для ...,Введение в разработку на .NET Framework.,1,0,1
1,2024-04-01 10:00:00,Основы программирования,Подробное объяснение логических операторов в J...,Нет.,Предоставить ещё больше практических задач пос...,Детальное изучение асинхронного программирован...,1,0,1
2,2024-04-01 15:00:00,Основы программирования,Введение в программирование на Python было про...,Нет.,"Улучшить качество видеоматериалов, иногда было...",Библиотеки Python для анализа данных и машинно...,1,0,1
3,2024-04-01 17:00:00,Основы программирования,"Понятное изложение основных понятий, хорошая с...","Да, циклы for и while были объяснены слишком б...",Добавить видеоматериалы для самостоятельного и...,Функциональное программирование.,1,0,1
4,2024-04-02 09:00:00,Основы программирования,Кларность и структура материала по основам Jav...,Нет.,Ввести больше примеров на работу с массивами и...,Работа с асинхронным кодом в JavaScript.,1,0,1


In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    103 non-null    object
 1   question_1   103 non-null    object
 2   question_2   103 non-null    object
 3   question_3   103 non-null    object
 4   question_4   103 non-null    object
 5   question_5   103 non-null    object
 6   is_relevant  103 non-null    int64 
 7   object       103 non-null    int64 
 8   is_positive  103 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 7.4+ KB


In [6]:
data_train['timestamp']

0      2024-04-01 08:30:00
1      2024-04-01 10:00:00
2      2024-04-01 15:00:00
3      2024-04-01 17:00:00
4      2024-04-02 09:00:00
              ...         
98     2024-04-24 10:22:00
99     2024-04-24 18:00:00
100    2024-04-25 20:00:00
101       27.04.2024:19:00
102       27.04.2024:19:00
Name: timestamp, Length: 103, dtype: object

In [7]:
data_train.iloc[101:]

,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive
101,27.04.2024:19:00,Java api браузеров,Понравилась тема,"Да, преподователь нес чушь и путался в своих ж...",Поменяйте преподователя,Хотел бы повторить тему Java коллекции,1,2,0
102,27.04.2024:19:00,Java api браузеров,Понравился методический вебинар,Преподователь сошел с ума,Я думаю с другим преподователем я получил бы б...,Хочу перепройти вебинар с другим преподователем,1,2,0


In [8]:
# data_train['timestamp'] = pd.to_datetime(data_train['timestamp'])

In [9]:
# data_train['time_of_day'] = pd.cut(data_train['timestamp'].dt.hour, bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])

In [10]:
data_train.head()

,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive
0,2024-04-01 08:30:00,Основы программирования,Интерактивный подход к изучению основ C# был о...,Нет.,Увеличить количество практических заданий для ...,Введение в разработку на .NET Framework.,1,0,1
1,2024-04-01 10:00:00,Основы программирования,Подробное объяснение логических операторов в J...,Нет.,Предоставить ещё больше практических задач пос...,Детальное изучение асинхронного программирован...,1,0,1
2,2024-04-01 15:00:00,Основы программирования,Введение в программирование на Python было про...,Нет.,"Улучшить качество видеоматериалов, иногда было...",Библиотеки Python для анализа данных и машинно...,1,0,1
3,2024-04-01 17:00:00,Основы программирования,"Понятное изложение основных понятий, хорошая с...","Да, циклы for и while были объяснены слишком б...",Добавить видеоматериалы для самостоятельного и...,Функциональное программирование.,1,0,1
4,2024-04-02 09:00:00,Основы программирования,Кларность и структура материала по основам Jav...,Нет.,Ввести больше примеров на работу с массивами и...,Работа с асинхронным кодом в JavaScript.,1,0,1


In [11]:
data_train["is_relevant"].value_counts()

is_relevant
1    90
0    13
Name: count, dtype: int64

In [12]:
data_train["object"].value_counts()

object
0    38
2    35
1    30
Name: count, dtype: int64

In [13]:
data_train["is_positive"].value_counts()

is_positive
1    83
0    20
Name: count, dtype: int64

## Надо подумать
1) Как разделить данные для теста и тренировки

## Данные для колонки `is_relevant`

In [14]:
class_list_relevant = sorted([str(val) for val in list(data_train['is_relevant'].unique())])

class_list_relevant

['0', '1']

## Данные для колонки `object`

In [15]:
class_list_object = sorted([str(val) for val in list(data_train['object'].unique())])

class_list_object

['0', '1', '2']

## Данные для колонки `is_positive`

In [16]:
class_list_positive = sorted([str(val) for val in list(data_train['is_positive'].unique())])
class_list_positive

['0', '1']

## Скрипты для лемантизации и очистки текста

In [17]:
s = "Привет 12344.#"
re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower())

'привет        '

In [18]:

def augmentation_data(data: pd.DataFrame, label: str = 'is_relevant'):
    augment_data = pd.DataFrame()

    col_number = [2, 3, 4, 5]

    # Генерация всех возможных комбинаций без повторений
    for len_comb in range(2, 5):
        # Комбинация колонок
        comb_list = list(permutations(col_number, len_comb))
        

        # Переберем все комбинации
        for comb in comb_list:

            data_buffer = pd.DataFrame()
            data_buffer["text"] = data[f'question_{comb[0]}'].copy()

            for col in comb[1:]:
                data_buffer['text'] = data_buffer['text'] + " " + data[f'question_{col}'].copy()
            
            data_buffer["label"] = data[label]
            
            if label != "object":
                # data_buffer = data_buffer[data_buffer["label"] == 0]
                data_buffer_neg = data_buffer[data_buffer["label"] == 0].copy()
                
                data_buffer_pos = resample(data_buffer[data_buffer["label"] == 1],
                     replace=False,
                     n_samples = data_buffer_neg.shape[0],
                     random_state=12345)
                
                data_buffer = pd.concat([data_buffer_neg, data_buffer_pos], ignore_index=True)
                # print(data_buffer["label"].value_counts())
            
            augment_data = pd.concat([augment_data, data_buffer], ignore_index=True)
    
    print(augment_data['label'].value_counts())
    return augment_data




def natasha_lemmant(input_str, segmenter, morph_tagger, syntax_parser,morph_vocab):

    doc = Doc(input_str)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    result = ' '.join([_.lemma for _ in doc.tokens])
    return result



def clear_string(s, stop_words): # Функция для очистки
    # print(re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split())
    # s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁa-zA-Z]', ' ', str(s).lower()).split() if not val in stop_words])
    s = ' '.join([val for val in re.sub(r'[^а-яА-ЯёЁ]', ' ', str(s).lower()).split() if not val in stop_words])
    return s


def create_dataset_spasy(data, label,  is_augment = False):
    stop_words = list(stopwords.words('russian'))
    
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    
    result = []
    
    if is_augment == True:
        data = augmentation_data(data, label)
    
    else:
        data['text'] = data['question_2'] + " " + data['question_3'] + " " + data['question_4'] + " " + data['question_5']
        data["label"] = data[label]
        
    data = json.loads(data.to_json(orient='records'))
    for d in tqdm(data):
        
        text = clear_string(d['text'], stop_words)
        if (text != '') and (text !=' '):
            text = natasha_lemmant(text, segmenter, morph_tagger, syntax_parser, morph_vocab)
    
            s = (text, {'cats': {str(d['label']): 1}})
            result.append(s)
    return result

## Разделяем данные на тренировочную и валидационную выборку

In [19]:
data_relevant_train, data_relevant_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['is_relevant']
)

data_relevant_train["is_relevant"].value_counts(), data_relevant_val["is_relevant"].value_counts()

(is_relevant
 1    60
 0     9
 Name: count, dtype: int64,
 is_relevant
 1    30
 0     4
 Name: count, dtype: int64)

In [20]:
data_object_train, data_object_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['object']
)

data_object_train["object"].value_counts(), data_object_val["object"].value_counts()

(object
 0    26
 2    23
 1    20
 Name: count, dtype: int64,
 object
 2    12
 0    12
 1    10
 Name: count, dtype: int64)

In [21]:
data_positive_train, data_positive_val = train_test_split(
    data_train,
    test_size=0.33,
    random_state=12345,
    stratify = data_train['is_positive']
)

data_positive_train["is_positive"].value_counts(), data_positive_val["is_positive"].value_counts()

(is_positive
 1    56
 0    13
 Name: count, dtype: int64,
 is_positive
 1    27
 0     7
 Name: count, dtype: int64)

## Очищаем данные

In [22]:
data_relevant_train = create_dataset_spasy(data_relevant_train, label = "is_relevant", is_augment = True)
data_relevant_val = create_dataset_spasy(data_relevant_val, label = "is_relevant")

data_object_train = create_dataset_spasy(data_object_train, label = "object", is_augment = True)
data_object_val = create_dataset_spasy(data_object_val, label = "object")

data_positive_train = create_dataset_spasy(data_positive_train, label = "is_positive", is_augment = True)
data_positive_val = create_dataset_spasy(data_positive_val, label = "is_positive")

label
0    540
1    540
Name: count, dtype: int64


100%|██████████| 34/34 [00:00<00:00, 44.59it/s]


label
0    1560
2    1380
1    1200
Name: count, dtype: int64


100%|██████████| 34/34 [00:00<00:00, 47.68it/s]


label
0    780
1    780
Name: count, dtype: int64


100%|██████████| 34/34 [00:00<00:00, 44.30it/s]


In [23]:
data_relevant_train[0]

('обсуждение ошибка код такой скучный составить список фильм просмотр каждый слово ошибка упоминаться',
 {'cats': {'0': 1}})

In [24]:
len(data_relevant_train), len(data_relevant_val)

(1080, 34)

In [25]:
len(data_object_train), len(data_object_val)

(4140, 34)

In [26]:
len(data_positive_train), len(data_positive_val)

(1560, 34)

In [27]:
list(data_object_train[0][1]["cats"].keys())[0]

'0'

In [28]:
max_len = 0
max_txt = ""
for val in (data_object_train + data_object_val + data_relevant_train + data_relevant_val + data_positive_train + data_positive_val):
    if len(val[0]) > max_len:
        max_len = len(val[0])
        max_txt = val[0]
        
max_len, max_txt

(608,
 'тема понравиться обсуждаться распространить среди аналитик инструмент рассказать рассказать предыдущий вопрос такой колаб описание интерфейс правило работа колаб нужный функция колабый библиотека колабнуть открыть колаб вижуал студио установить модуль открытие колабый вижуал студио такой момент раскрыть следующий тема такой колаб описание интерфейс правило работа колаб нужный функция колабый библиотека колабнуть открыть колаб вижуал студио установить модуль открытие колабый вижуал студио это первый прийти ум кстати зря прислать ссылка урок урок комментарий недовольный пользователь библиотека колабнуть')

In [29]:
# for i in range(len(data_object_train)):
#     if list(data_object_train[i][1]["cats"].keys())[0] == "2":
#         display(data_object_train[i])


## Создаём модель классификайци spacy

In [30]:
s = "обзор последний достижение область квантовый вычисление впечатляющий сложность тема требовать глубокий погружение организовать серия вебинар квантовый технология участие эксперт квантовый криптография она практический применение"
len(s)

228

In [31]:
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL



class TextClassifer:
    def __init__(self, label_list=None, show_info = True, name_model = "test", eval_metric = "roc_auc"):
#         spacy.prefer_gpu()
        
        self.name_model = name_model
        self.eval_metric = eval_metric
        
        # self.nlp = spacy.blank('xx')
        # self.nlp = spacy.load("ru_core_news_lg")
        self.nlp = spacy.blank('ru')
        self.nlp.add_pipe('tok2vec')
        
#         config = {
#             "model": {
#                 "@architectures": "spacy-transformers.TransformerModel.v3",
#                 "name": "ai-forever/sbert_large_mt_nlu_ru",
# #                 "tokenizer_config": {"use_fast": True},
# #                 "transformer_config": {"output_attentions": True},
# #                 "mixed_precision": True,
# #                 "grad_scaler_config": {"init_scale": 32768}
#             }
#         }
        
#         self.nlp.add_pipe('transformer', config = config)
        
#         self.nlp.add_pipe(
#             "hf_token_pipe", # "hf_text_pipe",
#             config={
#                 "model": "ai-forever/sbert_large_mt_nlu_ru",
#             },
#         )
        
        
        self.nlp.add_pipe('textcat', last = True)

        self.textcat = self.nlp.get_pipe('textcat')
#         print(spacy.prefer_gpu())
        
        self.label_list = label_list
        
        if label_list:
            for label in label_list:
                self.textcat.add_label(label)
        else:
            print('''
            Ошибка !!!
            Нужен список классов
            ''')
            
        if show_info:
            print(f'Pipline: {self.nlp.pipe_names}')
            print(self.nlp.analyze_pipes(pretty=True))
    
    def fit(self, epoch=10, data_train = None, batch_size=4, val = None):
        
        error = False # Была ли ошибка при подсчёте метрики
        
        # optimizer = self.nlp.begin_training()
        
        best_metrics = 0
        self.best_metrics_all = None
        
        other_pipes = [pipe for pipe in self.nlp.pipe_names if pipe not in ['textcat', "tok2vec"]]
        
        # __________________ Инициализация NLP __________________ 
        examples = []
        for text, annots in data_train:
            examples.append(Example.from_dict(self.nlp.make_doc(text), annots))
        self.nlp.initialize(lambda: examples)
        
        # optimizer = self.nlp.create_optimizer()
        # _______________________________________________________ 
        print(f"other_pipes: {other_pipes}")
        with self.nlp.disable_pipes(): #*other_pipes или []
            for epoch in tqdm(range(1, epoch+1)):
                random.shuffle(data_train)
                losses = {}
                for batch in spacy.util.minibatch(data_train, size=batch_size):
                    examples = []
                    
                    for text, annotations in batch:
                        # doc = self.nlp.make_doc(text)
                        # example = Example.from_dict(doc, annotations)
                        # examples.append(example)
                        examples.append(Example.from_dict(self.nlp.make_doc(text), annotations))
                        
                    self.nlp.update(examples, losses=losses, drop=0.1) # , sgd=optimizer
                    
                if val:
                    
                    metric = self.evaluate(val)
                    if best_metrics < metric[self.eval_metric]:
                        best_metrics = metric[self.eval_metric]
                        self.best_metrics_all = metric
                        
                        self.nlp.to_disk(f"/kaggle/working/best_model_{self.name_model}")
                    
                    print(metric)
                    print('Losses', losses)
                else:     
                    print('Losses', losses)
                    
                print("\n\n", "*" * 100, "\n\n")

#         self.nlp.to_disk('model_text_cat')
#         if not error: # Если ошибки не было 
        self.nlp = spacy.load(f"/kaggle/working/best_model_{self.name_model}")
        
        print('\n\n', "*"*100)
        print(f"best_metrics: {self.best_metrics_all}")
        print('\n\n', "*"*100)
        
    def predict(self, data):
        '''
        Функция определения нужного списка.

        :param data: .
        :type data: 
        
        :return: .
        :rtype: .
        '''
        
        self.nlp = spacy.load(f"/kaggle/working/best_model_{self.name_model}")

        return self.nlp
    
    def predict_proba(self, data):
        predict_proba = []
        
        for elem in data: # Пробегаемся по каждому элементу
            doc = self.nlp(elem[0]) #Обрабатываем текст
            
            arr_predict_proba = []
            for i, lbl in enumerate(self.label_list):
                arr_predict_proba.append(doc.cats[lbl])
            
            # Исправляем ошибку где сумма значений больше 1 или меньше 1
            try:
                sum_lst = sum(arr_predict_proba)
                new_lst = [el/sum_lst for el in arr_predict_proba]
                predict_proba.append(np.array(new_lst))    
            except:
                print(f'Входная строка <{elem[0]}>')
                print(f'Ошибка со значениями {doc.cats}')
                print(f'Предсказания:  {arr_predict_proba}')
            
        return np.array(predict_proba)
    
    def pr_auc_score(self, data):
        y_true = []
        
        for elem in data: # Пробегаемся по каждому элементу
            doc = self.nlp(elem[0]) #Обрабатываем текст
#             print(doc.cats)
            pred = max(doc.cats, key=doc.cats.get) # Класс
            true = list(elem[1].get('cats').keys())[0] # Класс
            
            for i, lbl in enumerate(self.label_list):
                if true == lbl:
                    y_true.append([0 for _ in range(i)] + [1] + [0 for _ in range(len(self.label_list) - i - 1)] )
                    break
        
        y_true = np.array(y_true)
        pr_proba = self.predict_proba(data)
        
        # print(f'y_true: {y_true}')
        # print(f'pr_proba: {pr_proba}')
        
        precision = dict()
        recall = dict()
        pr_auc = dict()
        for i, lbl in enumerate(self.label_list):
            precision[i], recall[i], _ = precision_recall_curve(y_true[:, i], pr_proba[:, i])
            pr_auc[lbl] = auc(recall[i], precision[i])

            
        # Построение кривой Precision-Recall AUC для каждого класса
        plt.figure()
#         plt.plot(recall["micro"], precision["micro"], label='micro-average (AUC = {0:0.2f})'.format(pr_auc["micro"]))
        for i, lbl in enumerate(self.label_list):
            plt.plot(recall[i], precision[i], label=f'class {lbl} (AUC = {round(pr_auc[lbl], 3)})')
            print(f'PR-AUC class {lbl} ({i}): ', pr_auc[lbl])

        # Настройка внешнего вида графика
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Precision-Recall AUC')
        plt.legend(loc="lower right")
        plt.figure(figsize=(20, 15))
        plt.show()
        
        
        
    
    def evaluate(self, data):
        '''
        Функция подсчёта метрики.

        :param data: Список кортежей в формате text_cat spacy.
        :type data: .
        
        :return: .
        :rtype: .
        '''
        count = len(data)
        count_true = 0
            
        true_labels = []
        
        # ('порт прибытия спб пкт спб пкт спб пкт', {'cats': {'POSITIVE': 1}}),

        for elem in data: # Пробегаемся по каждому элементу
            doc = self.nlp(elem[0]) #Обрабатываем текст
#             print(doc.cats)
            pred = max(doc.cats, key=doc.cats.get) # Класс
            true = list(elem[1].get('cats').keys())[0] # Класс
            
            arr_predict_proba = []
            for i, lbl in enumerate(self.label_list):
                if true == lbl:
                    true_labels.append(i)
            
            if pred == true:
                count_true += 1
        
        pr_proba = self.predict_proba(data)
        print(np.array(true_labels))
        print(np.argmax(np.array(pr_proba), axis = 1))
        # print(np.array(pr_proba)[:, 1])
        
        # если не мулитиклассовая классификация
        if len(self.label_list) == 2:
            roc_auc = roc_auc_score(np.array(true_labels), np.array(pr_proba)[:, 1])
            f1_metric = f1_score(np.array(true_labels), np.argmax(np.array(pr_proba), axis = 1))
            # print(f1_metric, np.argmax(np.array(pr_proba), axis = 1), np.array(pr_proba))
            
        # если мулитиклассовая классификация
        else:    
            roc_auc = roc_auc_score(np.array(true_labels), np.array(pr_proba), multi_class='ovr')
            f1_metric = f1_score(np.array(true_labels), np.argmax(np.array(pr_proba), axis = 1), average = 'micro')
        
        # self.pr_auc_score(data)
        return {'Accuracy': count_true/count, 'roc_auc': roc_auc, "f1_metric": f1_metric}

In [32]:
# try:
#     del model_relevant  #deleting the model 
# except:
#     pass

# # model will still be on cache until its place is taken by other objects so also execute the below lines
# import gc # garbage collect library
# import torch 
# gc.collect()
# torch.cuda.empty_cache() 

## Обучаем модель для relevant

In [33]:
model_relevant = TextClassifer(label_list = class_list_relevant, name_model = 'relevant', eval_metric = 'f1_metric')

Pipline: ['tok2vec', 'textcat']

============================= Pipeline Overview =============================

#   Component   Assigns      Requires   Scores            Retokenizes
-   ---------   ----------   --------   ---------------   -----------
0   tok2vec     doc.tensor                                False      
                                                                     
1   textcat     doc.cats                cats_score        False      
                                        cats_score_desc              
                                        cats_micro_p                 
                                        cats_micro_r                 
                                        cats_micro_f                 
                                        cats_macro_p                 
                                        cats_macro_r                 
                                        cats_macro_f                 
                                        cats_mac

In [34]:
"""
data_relevant_train
data_relevant_val 

data_object_train
data_object_val

data_positive_train
data_positive_val
"""

'\ndata_relevant_train\ndata_relevant_val \n\ndata_object_train\ndata_object_val\n\ndata_positive_train\ndata_positive_val\n'

Необходимо подбить метрику `0.9375`

In [35]:
model_relevant.fit(epoch = 30, data_train = data_relevant_train, batch_size = 4, val = data_relevant_val)

[2024-04-27 15:18:27,441] [INFO] Created vocabulary
[2024-04-27 15:18:27,442] [INFO] Finished initializing nlp object


other_pipes: []


  0%|          | 0/30 [00:00<?, ?it/s]

[1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1]


  3%|▎         | 1/30 [00:06<03:08,  6.49s/it]

...

 97%|█████████▋| 29/30 [02:57<00:06,  6.15s/it]

[1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1]
{'Accuracy': 0.9117647058823529, 'roc_auc': 0.775, 'f1_metric': 0.9491525423728815}
Losses {'tok2vec': 0.0, 'textcat': 7.1904416928037706e-12}


 **************************************************************************************************** 




100%|██████████| 30/30 [03:04<00:00,  6.13s/it]

[1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1]
{'Accuracy': 0.9117647058823529, 'roc_auc': 0.775, 'f1_metric': 0.9491525423728815}
Losses {'tok2vec': 0.0, 'textcat': 1.1843472474567653e-11}


 **************************************************************************************************** 






 ****************************************************************************************************
best_metrics: {'Accuracy': 0.9117647058823529, 'roc_auc': 0.7916666666666666, 'f1_metric': 0.9491525423728815}


 ****************************************************************************************************


## Обучаем модель для object
Надо побить рекорд `0.6764705882352942`

In [36]:
model_object = TextClassifer(label_list = class_list_object, name_model = 'object', eval_metric = 'f1_metric')

Pipline: ['tok2vec', 'textcat']

============================= Pipeline Overview =============================

#   Component   Assigns      Requires   Scores            Retokenizes
-   ---------   ----------   --------   ---------------   -----------
0   tok2vec     doc.tensor                                False      
                                                                     
1   textcat     doc.cats                cats_score        False      
                                        cats_score_desc              
                                        cats_micro_p                 
                                        cats_micro_r                 
                                        cats_micro_f                 
                                        cats_macro_p                 
                                        cats_macro_r                 
                                        cats_macro_f                 
                                        cats_mac

In [37]:
model_object.fit(epoch = 20, data_train = data_object_train, batch_size = 4, val = data_object_val)

[2024-04-27 15:21:33,199] [INFO] Created vocabulary
[2024-04-27 15:21:33,200] [INFO] Finished initializing nlp object


other_pipes: []


  0%|          | 0/20 [00:00<?, ?it/s]

[2 1 2 0 0 0 1 1 1 2 0 2 0 2 0 1 0 2 1 2 2 2 0 0 2 1 1 1 0 2 0 1 2 0]
[2 1 2 0 0 2 1 2 1 2 0 1 0 2 1 2 1 2 2 2 0 1 2 2 2 0 0 1 1 1 1 2 2 2]


  5%|▌         | 1/20 [00:25<08:09, 25.76s/it]

...

 95%|█████████▌| 19/20 [08:01<00:25, 25.44s/it]

[2 1 2 0 0 0 1 1 1 2 0 2 0 2 0 1 0 2 1 2 2 2 0 0 2 1 1 1 0 2 0 1 2 0]
[2 1 2 1 0 2 1 2 0 2 0 2 0 2 1 1 1 2 2 2 2 2 0 2 2 2 0 2 1 2 1 1 1 2]
{'Accuracy': 0.5588235294117647, 'roc_auc': 0.7487373737373737, 'f1_metric': 0.5588235294117647}
Losses {'tok2vec': 0.0, 'textcat': 2.035445266242665e-09}


 **************************************************************************************************** 




100%|██████████| 20/20 [08:27<00:00, 25.36s/it]

[2 1 2 0 0 0 1 1 1 2 0 2 0 2 0 1 0 2 1 2 2 2 0 0 2 1 1 1 0 2 0 1 2 0]
[2 1 2 1 0 2 1 2 0 2 0 2 0 2 1 1 1 2 2 2 2 2 0 2 2 2 0 2 1 2 1 1 1 2]
{'Accuracy': 0.5588235294117647, 'roc_auc': 0.7498737373737372, 'f1_metric': 0.5588235294117647}
Losses {'tok2vec': 0.0, 'textcat': 5.931927950445956e-11}


 **************************************************************************************************** 






 ****************************************************************************************************
best_metrics: {'Accuracy': 0.5882352941176471, 'roc_auc': 0.7054292929292929, 'f1_metric': 0.5882352941176471}


 ****************************************************************************************************


## Обучаем модель для positive
Надо побить `0.8928571428571429`

In [38]:
model_positive = TextClassifer(label_list = class_list_positive, name_model = 'positive', eval_metric = 'f1_metric')

Pipline: ['tok2vec', 'textcat']

============================= Pipeline Overview =============================

#   Component   Assigns      Requires   Scores            Retokenizes
-   ---------   ----------   --------   ---------------   -----------
0   tok2vec     doc.tensor                                False      
                                                                     
1   textcat     doc.cats                cats_score        False      
                                        cats_score_desc              
                                        cats_micro_p                 
                                        cats_micro_r                 
                                        cats_micro_f                 
                                        cats_macro_p                 
                                        cats_macro_r                 
                                        cats_macro_f                 
                                        cats_mac

In [39]:
pred = np.array([[0.5, 0.5], [0.2, 0.8]])
THRESHOLD = 0.25

np.where(pred > THRESHOLD, 1, 0)

array([[1, 1],
       [0, 1]])

In [40]:
model_positive.fit(epoch = 30, data_train = data_positive_train, batch_size = 4, val = data_positive_val)

[2024-04-27 15:30:02,248] [INFO] Created vocabulary
[2024-04-27 15:30:02,249] [INFO] Finished initializing nlp object


other_pipes: []


  0%|          | 0/30 [00:00<?, ?it/s]

[0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]


  3%|▎         | 1/30 [00:09<04:33,  9.44s/it]

...

 97%|█████████▋| 29/30 [04:14<00:08,  8.64s/it]

[0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]
[0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]
{'Accuracy': 0.8823529411764706, 'roc_auc': 0.9206349206349207, 'f1_metric': 0.92}
Losses {'tok2vec': 0.0, 'textcat': 1.901071255682683e-12}


 **************************************************************************************************** 




100%|██████████| 30/30 [04:22<00:00,  8.76s/it]

[0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]
[0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1]
{'Accuracy': 0.8823529411764706, 'roc_auc': 0.9206349206349207, 'f1_metric': 0.92}
Losses {'tok2vec': 0.0, 'textcat': 2.086276450397916e-12}


 **************************************************************************************************** 






 ****************************************************************************************************
best_metrics: {'Accuracy': 0.9411764705882353, 'roc_auc': 0.9576719576719577, 'f1_metric': 0.9615384615384615}


 ****************************************************************************************************


## Подсчитаем метрику общую на валидации
Надо побиить метрику - 0.835609243697479

In [41]:
(model_relevant.best_metrics_all['f1_metric'] + model_object.best_metrics_all['f1_metric'] + model_positive.best_metrics_all['f1_metric']) / 3

0.83297543267633

In [43]:
(model_relevant.best_metrics_all['roc_auc'] + model_object.best_metrics_all['roc_auc'] + model_positive.best_metrics_all['roc_auc']) / 3

0.818255972422639

## Сохраняем модели

In [42]:
%cd /kaggle/working/
!zip -r /kaggle/working/best_model_relevant.zip best_model_relevant

!zip -r /kaggle/working/best_model_object.zip best_model_object

!zip -r /kaggle/working/best_model_positive.zip best_model_positive

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: best_model_relevant/ (stored 0%)
  adding: best_model_relevant/config.cfg (deflated 62%)
  adding: best_model_relevant/textcat/ (stored 0%)
  adding: best_model_relevant/textcat/cfg (deflated 19%)
  adding: best_model_relevant/textcat/model (deflated 54%)
  adding: best_model_relevant/tokenizer (deflated 84%)
  adding: best_model_relevant/vocab/ (stored 0%)
  adding: best_model_relevant/vocab/vectors (deflated 45%)
  adding: best_model_relevant/vocab/lookups.bin (stored 0%)
  adding: best_model_relevant/vocab/key2row (stored 0%)
  adding: best_model_relevant/vocab/strings.json (deflated 88%)
  adding: best_model_relevant/vocab/vectors.cfg (stored 0%)
  adding: best_model_relevant/tok2vec/ (stored 0%)
  adding: best_model_relevant/tok2vec/cfg (stored 0%)
  adding: best_model_relevant/tok2vec/model (deflated 9%)
  adding: best_model_relevant/meta.json (deflated 54%)
  adding: best_model_object/ (stored 0%)
  adding: best_model_object/config.cfg (deflated 62%)
  adding: best_mod